## 1. Module Import

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

### < GPU check >

In [2]:
# 텐서플로우 GPU 사용 확인
import tensorflow as tf
tf.debugging.set_log_device_placement(True)

# 텐서 생성
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [7]:
# GPU 병렬 사용
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
  inputs = tf.keras.layers.Input(shape=(1,))
  predictions = tf.keras.layers.Dense(1)(inputs)
  model = tf.keras.models.Model(inputs=inputs, outputs=predictions)
  model.compile(loss='mse',
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.2))
    
# 하지만 이 기능을 사용하려면 코퍼스 덩어리 파일을 나눠서 적용시켜야함.

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhos

## 2. 대화 내용 추출 

### 2.1. 복수의 텍스트 통합

In [4]:
# 소설 txt 파일을 불러온후 list로 각자 변환 (with은 close 불필요)
with open('./현대소설 텍스트 1.txt', 'r', encoding = 'utf-8', ) as t1:
    text_list = []
    for line in t1:
        text_list.append(line)
    
with open('./현대소설 텍스트 2.txt', 'r', encoding = 'utf-8') as t2:
    text_list2 = []
    for line in t2:
        text_list2.append(line)
        
"""'
cp949' codec can't decode byte 0xb6 in position 20: illegal multibyte sequence ERROR가 발생하므로 
UTF-8 형식으로 변환해 불러오기
"""

# 리스트를 병합(결합) 후 txt 파일로 저장
with open('./현대소설 단순 결합본.txt', 'w', encoding = 'utf-8') as tt:
    tt.writelines(text_list)
    tt.writelines(text_list2)

### #2.2. 컬럼 분류

In [18]:

# 대화 내용이나 시간 정보, 유저 정보등을 제거하고 순수한 대화, 텍스트 내용만을 추출하기 위한 과정.
# 합쳐진 텍스트 판다스로 확인, 판다스 적용
text_df = pd.read_csv('./현대소설 단순 결합본.txt', sep = "\n", encoding = "utf-8")

# 제일 처음 소설 파일의 내용을 in으로 지정했음.
story_only = text_df['in']
print(type(story_only))
story_only

# <pd df을 txt로 저장하는 과정>
story_only.to_csv('현대 소설 내용 추출본.txt', sep = '\t', index = False)

# 확인
story_only = pd.read_csv('./현대 소설 내용 추출본.txt', delimiter = '\t')
print(type(story_only))
story_only.head(20)


<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


,in
0,1. 젊은 느티나무 / 강신재
1,1
2,그에게는 언제나 비누 냄새가 난다.
3,"아니, 그렇지는 않다. 언제나라고는 할 수 없다."
4,그가 학교에서 돌아와 욕실로 뛰어가서 물을 뒤집어쓰고 나오는 때면 비누 냄새가 난다...
5,티이샤쓰로 갈아입은 그는 성큼성큼 내 방으로 걸어 들어와 아무렇게나 안락의자에 주저...
6,｢무얼 해?｣
7,대개 이런 소리를 던진다.
8,그런 때에 그에게서 비누 냄새가 난다. 그리고 나는 나에게 가장 슬프고 괴로운 시간...
9,｢뭘해?｣


## 3. 한글 단어 이외 제거

In [15]:
import re

# 정규표현식을 이용하여 특수문자 및 지저분한 자음 제거
story_only["in"] = story_only["in"].str.replace(pat=r'[^\w,ㄱ-ㅎ,ㅏ-ㅣ]', repl=r'', regex=True)
story_only.head(20)

,in
0,1젊은느티나무강신재
1,1
2,그에게는언제나비누냄새가난다
3,"아니,그렇지는않다언제나라고는할수없다"
4,그가학교에서돌아와욕실로뛰어가서물을뒤집어쓰고나오는때면비누냄새가난다나는책상앞에돌아앉아서...
5,"티이샤쓰로갈아입은그는성큼성큼내방으로걸어들어와아무렇게나안락의자에주저앉든가,창가에팔꿈치..."
6,무얼해
7,대개이런소리를던진다
8,그런때에그에게서비누냄새가난다그리고나는나에게가장슬프고괴로운시간이다가온것을깨닫는다엷은비...
9,뭘해


In [21]:
# 영어 소문자, 대문자, 숫자, 한자 제거
story_only["in"] = story_only["in"].str.replace(pat=r'[a-zA-Z0-9/一-龥/]', repl=r'', regex=True)
story_only.head(20)

,in
0,젊은느티나무강신재
1,
2,그에게는언제나비누냄새가난다
3,"아니,그렇지는않다언제나라고는할수없다"
4,그가학교에서돌아와욕실로뛰어가서물을뒤집어쓰고나오는때면비누냄새가난다나는책상앞에돌아앉아서...
5,"티이샤쓰로갈아입은그는성큼성큼내방으로걸어들어와아무렇게나안락의자에주저앉든가,창가에팔꿈치..."
6,무얼해
7,대개이런소리를던진다
8,그런때에그에게서비누냄새가난다그리고나는나에게가장슬프고괴로운시간이다가온것을깨닫는다엷은비...
9,뭘해


In [22]:
# txt 파일로 저장
story_only.to_csv('현대소설 내용 추출 및 정리본.txt', sep = '\t', encoding = 'utf-8', index = False)

## 4. 한국어 임베딩

### 4.1. 전처리한 내용 list로 변환

In [5]:
"""
# <csv to list for tokenize>
import csv
text = open(r'./현대소설 내용 추출 및 정리본.txt', 'r', encoding = 'utf-8')
rdr = csv.reader(text, delimiter='\t')
rdw = list(rdr)
text.close()
"""

In [19]:
with open('./현대소설 내용 추출 및 정리본.txt', 'r', encoding = 'utf-8') as t3:
    text_list3 = []
    for line in t3:
        text_list3.append(line)

### 4.2. str 변환 및 코퍼스 분할

### 4.2.1. Okt

In [20]:
# <this is the second way to Tokenization>
from konlpy.tag import Okt
from gensim.models import word2vec

In [21]:
twitter = Okt()
corpus_text = []

# 텍스트 한줄씩 처리
for line in text_list3:
    # 형태소 분석
    malist = twitter.pos(line[0], norm=True, stem=True)
    c = []
    for word in malist:
        # 조사, 어미, 문장 부호는 제외하고 처리
        if not word[1] in ["Josa", "Eomi", "Punctuation"]:
            c.append(word[0])
    # 형태소 사이에 공백을 넣고 양쪽 공백은 삭제
    cl = (" ".join(c)).strip()
    corpus_text.append(cl)

KeyboardInterrupt: 

In [26]:
# may take long time
print(type(corpus_text))
print()
#print(corpus_text)

<class '_io.TextIOWrapper'>



In [8]:
# 코퍼스 추출본 저장
with open('./현대소설 말뭉치 추출본.txt', 'w', encoding = 'utf-8') as ct:
    ct.writelines(corpus_text)

ValueError: I/O operation on closed file.

### 4.2.2. Mecab

In [23]:
from eunjeon import Mecab 

# 내용 추출 및 정리본은 txt파일임
with open('./현대소설 내용 추출 및 정리본.txt', 'r', encoding = 'utf-8', ) as t4:
    text_lsit_4_2_2 = []
    for line in t4:
            text_lsit_4_2_2.append(line)
            
str_corpus_mecab = (' '.join(text_lsit_4_2_2))

mecab = Mecab() 
mecab.nouns(str_corpus_mecab)

['느티나무',
 '강신재',
 '그',
 '비누',
 '냄새',
 '수',
 '그',
 '학교',
 '욕실',
 '물',
 '때',
 '비누',
 '냄새',
 '나',
 '책상',
 '앞',
 '그',
 '것',
 '그',
 '표정',
 '기분',
 '수',
 '티',
 '샤쓰',
 '그',
 '내방',
 '나',
 '안락의자',
 '창가',
 '팔꿈치',
 '나',
 '무얼',
 '소리',
 '때',
 '그',
 '비누',
 '냄새',
 '나',
 '나',
 '시간',
 '것',
 '비누',
 '향료',
 '가슴',
 '속',
 '것',
 '말',
 '것',
 '뭘',
 '해',
 '한마디',
 '그',
 '눈',
 '내',
 '얼굴',
 '눈동자',
 '내',
 '표정',
 '것',
 '그',
 '나',
 '것',
 '자신',
 '기분',
 '것',
 '편',
 '나',
 '나',
 '슬픔',
 '괴롬',
 '대로',
 '지혜',
 '일점',
 '응집',
 '순간',
 '그',
 '눈',
 '속',
 '응시',
 '수',
 '나',
 '것',
 '그',
 '눈',
 '속',
 '내',
 '무엇',
 '하루',
 '하룻밤',
 '사이',
 '바위',
 '파도',
 '소리',
 '가슴',
 '가지',
 '상념',
 '나',
 '일순',
 '전신',
 '되풀이',
 '애',
 '나',
 '수',
 '그',
 '눈',
 '의미',
 '수',
 '나',
 '괴롬',
 '슬픔',
 '것',
 '가슴',
 '속',
 '것',
 '다음',
 '찰나',
 '나',
 '만나',
 '자연',
 '위치',
 '그',
 '누이동생',
 '표면',
 '무시',
 '안정',
 '나',
 '위치',
 '것',
 '나',
 '그',
 '듯이',
 '어투',
 '경우',
 '것',
 '얼마',
 '추태',
 '나',
 '내',
 '목소리',
 '그',
 '무언지',
 '마음',
 '듯이',
 '고단',
 '거',
 '다리',
 '기지개',
 '대답',
 '성화',
 '영작',
 '숙제'

In [27]:
print(type(str_corpus_mecab))

<class 'str'>


### <코퍼스 파일 불러와 작업>

In [33]:
# Error: field larger than field limit (131072) 를 해결하기 위해 csv 수용 사이즈 확장
import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)
        
#==============================================================================================

# 말뭉치 분리 작업 시간 소요 없이 txt 파일을 불러와 바로 학습 시작
corpus_text = open(r'./현대소설 말뭉치 추출본.txt', 'r', encoding = 'utf-8')

rdr2 = csv.reader(corpus_text, delimiter='\t')
rdw2 = list(rdr2)
corpus_text.close()

print(type(rdw2), len(rdw2))

<class 'list'> 1


In [34]:
# ERROR sequence item 0: expected str instance, list found 를 해결하기 위해 str로 반복 변환하여 해결
for i in range(len(rdw2)):
    rdw2[i] = str(rdw2[i])

# list를 토큰화를 위해 str로 변환 후 확인
str_corpus = (' '.join(rdw2))

print(type(str_corpus))
print(len(str_corpus))
print(str_corpus)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### 4.3. 토큰화 진행

In [40]:
# Mecab 으로 형태소 구분한 str파일을 입력

In [36]:
tok = Tokenizer()
tok.fit_on_texts([str_corpus_mecab])

vocSize=len(tok.word_index)+1

seqs = list()

for word in str_corpus_mecab.split("\n"):
    encoded = tok.texts_to_sequences([word])[0]
    for i in range(1, len(encoded)):
        seq = encoded[:i+1]
        seqs.append(seq)
        
print(type(seqs))
print(seqs)

maxLen=max(len(i) for i in seqs)

seqs=pad_sequences(seqs ,maxlen=maxLen, padding="pre") 
seqs 

seqs = np.array(seqs)
x = seqs[:, :-1]

from tensorflow.keras.utils import to_categorical
y = seqs[:,-1]
y = to_categorical(y, num_classes=vocSize)         # 원-핫 인코딩 적용


<class 'list'>
[[4, 628], [630, 631], [6, 635], [636, 637], [642, 643], [642, 643, 644], [642, 643, 644, 645], [647, 648], [651, 652], [2, 654], [656, 657], [656, 657, 658], [58, 186], [663, 664], [666, 667], [670, 671], [670, 671, 672], [4, 676], [4, 676, 677], [6, 678], [4, 682], [690, 691], [701, 702], [295, 703], [707, 708], [707, 708, 709], [717, 718], [717, 718, 719], [733, 734], [733, 734, 735], [733, 734, 735, 736], [733, 734, 735, 736, 737], [733, 734, 735, 736, 737, 738], [739, 740], [739, 740, 741], [751, 752], [765, 766], [765, 766, 767], [765, 766, 767, 768], [765, 766, 767, 768, 769], [770, 771], [773, 774], [773, 774, 775], [781, 782], [783, 784], [785, 786], [789, 790], [791, 792], [796, 797], [796, 797, 798], [799, 800], [808, 809], [808, 809, 810], [808, 809, 810, 2], [814, 815], [818, 819], [818, 819, 820], [818, 819, 820, 821], [818, 819, 820, 821, 822], [823, 824], [838, 839], [296, 840], [841, 842], [6, 844], [6, 844, 845], [854, 855], [854, 855, 856], [854, 855, 

## 5. 모델 생성 및 훈련

### 5.1. 모듈 임포트 및 Layers 설계

In [37]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import Embedding
import tensorflow as tf
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.initializers import RandomNormal, Constant

In [38]:
model = Sequential()
model.add(Embedding(vocSize, 10, input_length=maxLen-1,))
model.add(tf.keras.layers.Dropout(0.3))
model.add(BatchNormalization())
model.add(LSTM(32))
model.add(BatchNormalization(
        momentum=0.99,
        epsilon=0.005,
        beta_initializer=RandomNormal(mean=0.0, stddev=0.05), 
        gamma_initializer=Constant(value=0.9)
    ))
model.add(Dense(vocSize, activation="softmax"))
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="Nadam")
model.summary()

"""
'Cannot convert a symbolic Tensor (lstm_1/strided_slice:0) to a numpy array.' ERROR 발생
Numpy 버전이 너무 높아서 Tensorflow에서 구현을 못한 것으로 numpy 다운그레이드로 해결 가능.
"""

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 226, 10)           382570    
_________________________________________________________________
dropout_1 (Dropout)          (None, 226, 10)           0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 226, 10)           40        
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                5504      
_________________________________________________________________
batch_normalization_3 (Batch (None, 32)                128       
_________________________________________________________________
dense_1 (Dense)              (None, 38257)             1262481   
Total params: 1,650,723
Trainable params: 1,650,639
Non-trainable params: 84
___________________________________________

"\n'Cannot convert a symbolic Tensor (lstm_1/strided_slice:0) to a numpy array.' ERROR 발생\nNumpy 버전이 너무 높아서 Tensorflow에서 구현을 못한 것으로 numpy 다운그레이드로 해결 가능.\n"

### 5.2. 학습 및 결과 출력

In [39]:
model.fit(x, y, epochs=100)

def sentGen(model, tok, word, n):
    sent = ""
    word2 = word
    for _ in range(n):
        encoded = tok.texts_to_sequences([word])[0]
        encoded = pad_sequences([encoded], maxlen=7, padding="pre")
        res = model.predict_classes(encoded)
        
        for w, i in tok.word_index.items():
            if i ==res:
                break
        word = word + " " + w
        sent = sent + " " + w
    
    sent = word2 + sent
    return sent

print('\n'*5)
result = sentGen(model, tok, "대화", 5)
print('유저 입력 단어 : {0}\n예상 후첨 단어 : {1}'.format(result[0:2], result[3:5]))
print('자동 추천 목록 : {}'.format(result))

Epoch 1/100
540/540 [==============================] - 124s 229ms/step - loss: 10.5943 - accuracy: 6.9529e-04
Epoch 2/100
540/540 [==============================] - 123s 228ms/step - loss: 9.7499 - accuracy: 0.0019
Epoch 3/100
540/540 [==============================] - 117s 217ms/step - loss: 8.9876 - accuracy: 0.0031
Epoch 4/100
 40/540 [=>............................] - ETA: 2:05 - loss: 8.0870 - accuracy: 0.0219

KeyboardInterrupt: 